In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
from torchvision import datasets, transforms

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [5]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
trans = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,transform=trans),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=trans),
    batch_size=batch_size, shuffle=True, **kwargs)


In [38]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [60]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.do = nn.Dropout(0.1)
        self.conv1 = nn.Conv2d(1, 12, 3, padding=1, bias=False) #RFi-1 RFo-3
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(12, 16, 3, padding=1, bias=False) #RFi-3 RFo-5
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2) #RFi-5 RFo-6
        self.conv3 = nn.Conv2d(16, 22, 3, padding=1, bias=False) #Rfi-6 Rfo-10
        self.bn3 = nn.BatchNorm2d(22)
        self.conv4 = nn.Conv2d(22, 26, 3, padding=1, bias=False) #Rfi-10 Rfo-14
        self.bn4 = nn.BatchNorm2d(26)
        self.pool2 = nn.MaxPool2d(2, 2) #Rfi-14 Rfo-16
        self.conv5 = nn.Conv2d(26, 32, 3, padding=1, bias=False) #Rfi-16 Rfo-24
        self.bn5 = nn.BatchNorm2d(32)
        self.conv6 = nn.Conv2d(32, 10, 1, bias=False) #Rfi-24 Rfo-24 #ANTMAN
        self.gap = nn.AvgPool2d(7,1) #GAP

    def forward(self, x):
        x = self.do(self.bn1(F.relu(self.conv1(x))))
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.pool1(x)
        x = self.do(self.bn3(F.relu(self.conv3(x))))
        x = self.bn4(F.relu(self.conv4(x)))
        x = self.pool2(x)
        x = self.gap(F.relu(self.conv6(self.bn5(F.relu(self.conv5(x))))))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [61]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 28, 28]             108
       BatchNorm2d-2           [-1, 12, 28, 28]              24
           Dropout-3           [-1, 12, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,728
       BatchNorm2d-5           [-1, 16, 28, 28]              32
         MaxPool2d-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 22, 14, 14]           3,168
       BatchNorm2d-8           [-1, 22, 14, 14]              44
           Dropout-9           [-1, 22, 14, 14]               0
           Conv2d-10           [-1, 26, 14, 14]           5,148
      BatchNorm2d-11           [-1, 26, 14, 14]              52
        MaxPool2d-12             [-1, 26, 7, 7]               0
           Conv2d-13             [-1, 32, 7, 7]           7,488
      BatchNorm2d-14             [-1, 3

<ipython-input-60-c5d21f19ba07>:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [62]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-60-c5d21f19ba07>:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.04466840997338295 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.59it/s]



Test set: Average loss: 0.0632, Accuracy: 9801/10000 (98.01%)



loss=0.03505292162299156 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.52it/s]



Test set: Average loss: 0.0560, Accuracy: 9800/10000 (98.00%)



loss=0.015697510913014412 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.10it/s]


...


Test set: Average loss: 0.0203, Accuracy: 9940/10000 (99.40%)



loss=0.0015035736141726375 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.01it/s]



Test set: Average loss: 0.0195, Accuracy: 9940/10000 (99.40%)



loss=0.002266551833599806 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 27.64it/s]



Test set: Average loss: 0.0228, Accuracy: 9934/10000 (99.34%)

